### Victor Rene Pérez Mayen.
### 19005180

### Space Apps Challenge 2019

### Planets Near and Far

#### Chasers of the Lost Data
##### https://2019.spaceappschallenge.org/challenges/planets-near-and-far/raiders-lost-data/details

In [49]:
import os
from sklearn.impute import SimpleImputer
import impyute as impy
import numpy as np, pandas as pd, tensorflow as tf, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [2]:
df_1 = pd.read_csv("NASA/Fireball_And_Bolide_Reports.csv")
df_2 = pd.read_csv("NASA/GLC03122015.csv")
df_3 = pd.read_csv("NASA/Meteorite_Landings.csv")
df_4 = pd.read_csv("NASA/Near-Earth_Comets_-_Orbital_Elements.csv")

### Fireball And Bolide Reports

In [3]:
df_1 #Fireball_And_Bolide_Reports

,Date/Time - Peak Brightness (UT),Latitude (Deg),Longitude (Deg),Altitude (km),Velocity (km/s),Velocity Components (km/s): vx,Velocity Components (km/s): vy,Velocity Components (km/s): vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,11/21/2009 08:53:00 PM,22.0S,29.2E,38.0,32.1,3.0,-17.0,-27.0,10000000000000,18.000
1,01/09/2015 10:41:11 AM,2.0N,28.8E,36.0,NaN,-10.7,-7.6,11.6,139000000000,0.410
2,05/16/2014 12:42:48 PM,44.2S,176.2W,44.0,NaN,14.4,4.6,6.5,309000000000,0.820
3,08/23/2014 06:29:41 AM,61.7S,132.6E,22.2,16.2,-2.3,5.7,16.5,3820000000000,7.600
4,12/12/2014 06:48:11 AM,33.5N,144.9E,26.3,NaN,11.5,-2.8,-2.2,33000000000,0.110
5,07/29/2014 07:38:07 AM,49.2S,172.2W,NaN,NaN,NaN,NaN,NaN,73000000000,0.230
6,06/01/2013 10:49:48 PM,65.6S,138.4E,28.9,NaN,NaN,NaN,NaN,34000000000,0.120
7,02/17/2015 01:19:50 PM,8.0S,11.2W,39.0,NaN,-28.2,3.4,4.6,33000000000,0.110
8,10/17/2014 02:07:36 PM,4.6S,66.3W,39.0,NaN,NaN,NaN,NaN,72000000000,0.230
9,12/08/2013 03:10:09 AM,32.8N,165.1W,23.5,NaN,2.3,2.5,-11.3,64000000000,0.200


In [4]:
#Se buscan los Null
np.sum(df_1.isnull())

Date/Time - Peak Brightness (UT)        0
Latitude (Deg)                          0
Longitude (Deg)                         0
Altitude (km)                          24
Velocity (km/s)                        85
Velocity Components (km/s): vx         40
Velocity Components (km/s): vy         40
Velocity Components (km/s): vz         40
Total Radiated Energy (J)               0
Calculated Total Impact Energy (kt)     0
dtype: int64

In [5]:
print(df_1["Altitude (km)"].head(5),df_1["Velocity (km/s)"].head(5),df_1["Velocity Components (km/s): vx"].head(5),
      df_1["Velocity Components (km/s): vy"].head(5),df_1["Velocity Components (km/s): vz"].head(5))

0    38.0
1    36.0
2    44.0
3    22.2
4    26.3
Name: Altitude (km), dtype: float64 0    32.1
1     NaN
2     NaN
3    16.2
4     NaN
Name: Velocity (km/s), dtype: float64 0     3.0
1   -10.7
2    14.4
3    -2.3
4    11.5
Name: Velocity Components (km/s): vx, dtype: float64 0   -17.0
1    -7.6
2     4.6
3     5.7
4    -2.8
Name: Velocity Components (km/s): vy, dtype: float64 0   -27.0
1    11.6
2     6.5
3    16.5
4    -2.2
Name: Velocity Components (km/s): vz, dtype: float64


In [6]:
df_1_nn = df_1.dropna(how = "any")
np.sum(df_1_nn.isnull())

Date/Time - Peak Brightness (UT)       0
Latitude (Deg)                         0
Longitude (Deg)                        0
Altitude (km)                          0
Velocity (km/s)                        0
Velocity Components (km/s): vx         0
Velocity Components (km/s): vy         0
Velocity Components (km/s): vz         0
Total Radiated Energy (J)              0
Calculated Total Impact Energy (kt)    0
dtype: int64

In [7]:
print("dimensiones con nulls =", df_1.shape)
print("dimensiones sin nulls =", df_1_nn.shape)

dimensiones con nulls = (92, 10)
dimensiones sin nulls = (7, 10)


In [8]:
df_1_nn

,Date/Time - Peak Brightness (UT),Latitude (Deg),Longitude (Deg),Altitude (km),Velocity (km/s),Velocity Components (km/s): vx,Velocity Components (km/s): vy,Velocity Components (km/s): vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,11/21/2009 08:53:00 PM,22.0S,29.2E,38.0,32.1,3.0,-17.0,-27.0,10000000000000,18.00
3,08/23/2014 06:29:41 AM,61.7S,132.6E,22.2,16.2,-2.3,5.7,16.5,3820000000000,7.60
14,12/25/2010 11:24:00 PM,38.0N,158.0E,26.0,18.1,18.0,-2.0,-4.0,20000000000000,33.00
23,02/15/2013 03:20:33 AM,54.8N,61.1E,23.3,18.6,12.8,-13.3,-2.4,375000000000000,440.00
25,10/08/2009 02:57:00 AM,4.2S,120.6E,19.1,19.2,14.0,-16.0,-6.0,20000000000000,33.00
26,06/28/2014 02:40:07 AM,18.9N,141.2E,26.3,12.4,12.0,3.5,-10.5,245000000000,0.67
47,05/16/2014 08:06:28 PM,39.4S,95.9W,30.8,18.0,2.9,13.4,-12.5,136000000000,0.40


##### As we can see, the parameter Velocity have a lot of nulls, but this value is the magnitude of the other velocities, so is a better idea drop only the rows that have only nulls.

In [9]:
df_1_ns = df_1.dropna(how = "all")
print(np.sum(df_1_ns.isnull()))
print("dimensiones sin nulls =", df_1_ns.shape)

Date/Time - Peak Brightness (UT)        0
Latitude (Deg)                          0
Longitude (Deg)                         0
Altitude (km)                          24
Velocity (km/s)                        85
Velocity Components (km/s): vx         40
Velocity Components (km/s): vy         40
Velocity Components (km/s): vz         40
Total Radiated Energy (J)               0
Calculated Total Impact Energy (kt)     0
dtype: int64
dimensiones sin nulls = (92, 10)


In [10]:
df_1_ss = df_1.dropna(subset = ["Velocity (km/s)","Velocity Components (km/s): vx",
                               "Velocity Components (km/s): vy","Velocity Components (km/s): vz"],how = "all")
print(np.sum(df_1_ss.isnull()))
print("dimensiones sin nulls =", df_1_ss.shape)

Date/Time - Peak Brightness (UT)        0
Latitude (Deg)                          0
Longitude (Deg)                         0
Altitude (km)                           2
Velocity (km/s)                        45
Velocity Components (km/s): vx          0
Velocity Components (km/s): vy          0
Velocity Components (km/s): vz          0
Total Radiated Energy (J)               0
Calculated Total Impact Energy (kt)     0
dtype: int64
dimensiones sin nulls = (52, 10)


In [11]:
v = round(np.sqrt(np.power(df_1["Velocity Components (km/s): vx"],2) + np.power(df_1["Velocity Components (km/s): vy"],2) + 
            np.power(df_1["Velocity Components (km/s): vz"],2)),1)
print(v)

0     32.0
1     17.5
2     16.5
3     17.6
4     12.0
      ... 
87     NaN
88     NaN
89    18.5
90    21.3
91    17.1
Length: 92, dtype: float64


In [25]:
#relative error in velocity
print(abs(df_1["Velocity (km/s)"]-v).dropna(how = "any"))

0     0.1
3     1.4
14    0.4
23    0.0
25    2.9
26    3.9
47    0.6
dtype: float64


In [27]:
df_1["Velocity (km/s)"] = v
print(df_1)

   Date/Time - Peak Brightness (UT) Latitude (Deg) Longitude (Deg)  \
0            11/21/2009 08:53:00 PM          22.0S           29.2E   
1            01/09/2015 10:41:11 AM           2.0N           28.8E   
2            05/16/2014 12:42:48 PM          44.2S          176.2W   
3            08/23/2014 06:29:41 AM          61.7S          132.6E   
4            12/12/2014 06:48:11 AM          33.5N          144.9E   
..                              ...            ...             ...   
87           05/04/2012 09:54:49 PM          76.7N           10.6W   
88           04/23/2012 10:01:10 PM          36.2N          107.4E   
89           07/25/2012 07:48:20 AM          36.4N           41.5E   
90           04/21/2015 01:42:51 AM          37.7N           39.6W   
91           05/15/2012 11:04:17 AM          61.8S          135.5W   

    Altitude (km)  Velocity (km/s)  Velocity Components (km/s): vx  \
0            38.0             32.0                             3.0   
1            36.0  

In [28]:
df_1_ss = df_1.dropna(subset = ["Velocity (km/s)","Velocity Components (km/s): vx",
                               "Velocity Components (km/s): vy","Velocity Components (km/s): vz"],how = "all")
print(np.sum(df_1_ss.isnull()))
print("dimensiones sin nulls =", df_1_ss.shape)

Date/Time - Peak Brightness (UT)       0
Latitude (Deg)                         0
Longitude (Deg)                        0
Altitude (km)                          2
Velocity (km/s)                        0
Velocity Components (km/s): vx         0
Velocity Components (km/s): vy         0
Velocity Components (km/s): vz         0
Total Radiated Energy (J)              0
Calculated Total Impact Energy (kt)    0
dtype: int64
dimensiones sin nulls = (52, 10)


In [41]:
df_1_ss

,Date/Time - Peak Brightness (UT),Latitude (Deg),Longitude (Deg),Altitude (km),Velocity (km/s),Velocity Components (km/s): vx,Velocity Components (km/s): vy,Velocity Components (km/s): vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,11/21/2009 08:53:00 PM,22.0S,29.2E,38.0,32.0,3.0,-17.0,-27.0,10000000000000,18.000
1,01/09/2015 10:41:11 AM,2.0N,28.8E,36.0,17.5,-10.7,-7.6,11.6,139000000000,0.410
2,05/16/2014 12:42:48 PM,44.2S,176.2W,44.0,16.5,14.4,4.6,6.5,309000000000,0.820
3,08/23/2014 06:29:41 AM,61.7S,132.6E,22.2,17.6,-2.3,5.7,16.5,3820000000000,7.600
4,12/12/2014 06:48:11 AM,33.5N,144.9E,26.3,12.0,11.5,-2.8,-2.2,33000000000,0.110
7,02/17/2015 01:19:50 PM,8.0S,11.2W,39.0,28.8,-28.2,3.4,4.6,33000000000,0.110
9,12/08/2013 03:10:09 AM,32.8N,165.1W,23.5,11.8,2.3,2.5,-11.3,64000000000,0.200
12,11/04/2014 08:13:30 PM,43.1N,115.8E,22.2,16.0,-7.2,-12.1,-7.7,156000000000,0.450
14,12/25/2010 11:24:00 PM,38.0N,158.0E,26.0,18.5,18.0,-2.0,-4.0,20000000000000,33.000
15,01/07/2015 01:05:59 AM,45.7N,26.9E,45.5,35.7,-35.4,1.8,-4.4,136000000000,0.400


In [48]:
#df_1_ss["Altitude (km)"].apply(lambda x: x.fillna(x.mean()))    
df_1_clear = df_1_ss.fillna(df_1_ss["Altitude (km)"].mean())
df_1_clear

,Date/Time - Peak Brightness (UT),Latitude (Deg),Longitude (Deg),Altitude (km),Velocity (km/s),Velocity Components (km/s): vx,Velocity Components (km/s): vy,Velocity Components (km/s): vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,11/21/2009 08:53:00 PM,22.0S,29.2E,38.000,32.0,3.0,-17.0,-27.0,10000000000000,18.000
1,01/09/2015 10:41:11 AM,2.0N,28.8E,36.000,17.5,-10.7,-7.6,11.6,139000000000,0.410
2,05/16/2014 12:42:48 PM,44.2S,176.2W,44.000,16.5,14.4,4.6,6.5,309000000000,0.820
3,08/23/2014 06:29:41 AM,61.7S,132.6E,22.200,17.6,-2.3,5.7,16.5,3820000000000,7.600
4,12/12/2014 06:48:11 AM,33.5N,144.9E,26.300,12.0,11.5,-2.8,-2.2,33000000000,0.110
7,02/17/2015 01:19:50 PM,8.0S,11.2W,39.000,28.8,-28.2,3.4,4.6,33000000000,0.110
9,12/08/2013 03:10:09 AM,32.8N,165.1W,23.500,11.8,2.3,2.5,-11.3,64000000000,0.200
12,11/04/2014 08:13:30 PM,43.1N,115.8E,22.200,16.0,-7.2,-12.1,-7.7,156000000000,0.450
14,12/25/2010 11:24:00 PM,38.0N,158.0E,26.000,18.5,18.0,-2.0,-4.0,20000000000000,33.000
15,01/07/2015 01:05:59 AM,45.7N,26.9E,45.500,35.7,-35.4,1.8,-4.4,136000000000,0.400


### #GLC03122015

In [12]:
df_2 #GLC03122015

,the_geom,OBJECTID,id,date_,time_,country,nearest_pl,hazard_typ,landslide_,trigger,...,population,countrycod,continentc,key_,version,user_id,tstamp,changeset_,latitude,longitude
0,POINT (-73.40200000000006 41.55850000000008),1,3177,03/07/2011 08:00:00 AM +0000,12/30/1899 08:00:00 AM +0000,United States,Grove Street from Anderson Avenue to Hine Hill...,landslide,Mudslide,Downpour,...,6523,NaN,NaN,US,1,1,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1,41.5585,-73.4020
1,POINT (113.91710000000012 0.1115000000000351),2,490,04/01/2008 07:00:00 AM +0000,NaN,Indonesia,"Borneo, Muara",landslide,Landslide,Rain,...,0,NaN,AS,ID,1,1,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1,0.1115,113.9171
2,POINT (-127.69800000000009 52.35450000000008),3,6760,01/30/2015 08:00:00 AM +0000,12/30/1899 08:00:00 AM +0000,NaN,"Ocean Falls, B.C.",landslide,Mudslide,Rain,...,8987,NaN,NaN,CA,2,7,Tue Feb 17 2015 15:42:41 GMT+0000 (UTC),3910846556,52.3545,-127.6980
3,POINT (-127.50620000000009 50.70530000000013),4,2494,09/24/2010 07:00:00 AM +0000,NaN,Canada,"road to Holberg, 3 km from hwy 19, Vancouver I...",landslide,Complex,Downpour,...,33430,NaN,NaN,CA,1,1,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1,50.7053,-127.5062
4,POINT (-132.41490000000013 53.331900000000125),5,6415,08/31/2014 07:00:00 AM +0000,12/30/1899 08:00:00 AM +0000,NaN,Rennell Sound Road,landslide,Mudslide,Downpour,...,14708,NaN,NaN,CA,1,7,Thu Dec 04 2014 15:14:07 GMT+0000 (UTC),1280292118,53.3319,-132.4149
5,POINT (-127.48460000000007 50.43350000000009),6,2493,09/24/2010 07:00:00 AM +0000,12/30/1899 08:00:00 AM +0000,Canada,"main road in Port Alice and Neucel Pulp Mill, ...",landslide,Mudslide,Downpour,...,33430,NaN,NaN,CA,1,1,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1,50.4335,-127.4846
6,POINT (-143.1680000000001 61.97800000000013),7,5194,07/25/2013 07:00:00 AM +0000,NaN,United States,"Wrangell-St. Elias National Preserve, Chisana, Ak",landslide,Landslide,Snowfall_snowmelt,...,1258,NaN,NaN,US,2,7,Fri Sep 12 2014 21:10:07 GMT+0000 (UTC),1985434843,61.9780,-143.1680
7,POINT (-127.0961000000001 50.5854000000001),8,4066,11/25/2011 08:00:00 AM +0000,NaN,Canada,"Fort McNeill, Vancover Island, British Colombia",landslide,Landslide,Downpour,...,33430,NaN,NaN,CA,1,1,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1,50.5854,-127.0961
8,POINT (-126.40390000000009 50.99750000000011),9,2505,09/26/2010 07:00:00 AM +0000,NaN,Canada,"Kingcome Inlet, ON",landslide,Mudslide,Downpour,...,33430,NaN,NaN,CA,1,1,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1,50.9975,-126.4039
9,POINT (116.38160000000016 1.3542000000000498),10,945,12/22/2008 08:00:00 AM +0000,NaN,Indonesia,"four villages in Lempake Jaya, North Samarinda...",landslide,Landslide,Downpour,...,0,NaN,AS,ID,1,1,Tue Apr 01 2014 00:00:00 GMT+0000 (UTC),1,1.3542,116.3816


In [15]:
#Se buscan los Null
np.sum(df_2.isnull())

the_geom         0
OBJECTID         0
id               0
date_            4
time_         5519
country        608
nearest_pl      64
hazard_typ       0
landslide_       0
trigger          0
storm_name    6323
fatalities       0
injuries         0
source_nam    4293
source_lin     505
location_a       0
landslide1       2
photos_lin    6499
cat_src          1
cat_id           0
countrynam       0
near             1
distance         0
adminname1      71
adminname2    2994
population       0
countrycod    6788
continentc    2171
key_             2
version          0
user_id          0
tstamp           0
changeset_       0
latitude         0
longitude        0
dtype: int64

### Meteorite Landings

In [13]:
df_3 #Meteorite_Landings

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)"
5,Adhi Kot,379,Valid,EH4,4239.0,Fell,01/01/1919 12:00:00 AM,32.10000,71.80000,"(32.1, 71.8)"
6,Adzhi-Bogdo (stone),390,Valid,LL3-6,910.0,Fell,01/01/1949 12:00:00 AM,44.83333,95.16667,"(44.83333, 95.16667)"
7,Agen,392,Valid,H5,30000.0,Fell,01/01/1814 12:00:00 AM,44.21667,0.61667,"(44.21667, 0.61667)"
8,Aguada,398,Valid,L6,1620.0,Fell,01/01/1930 12:00:00 AM,-31.60000,-65.23333,"(-31.6, -65.23333)"
9,Aguila Blanca,417,Valid,L,1440.0,Fell,01/01/1920 12:00:00 AM,-30.86667,-64.55000,"(-30.86667, -64.55)"


In [16]:
#Se buscan los Null
np.sum(df_3.isnull())

name              0
id                0
nametype          0
recclass          0
mass (g)        131
fall              0
year            291
reclat         7315
reclong        7315
GeoLocation    7315
dtype: int64

In [18]:
df_3_nn = df_3.dropna(how = "any")
np.sum(df_3_nn.isnull())

name           0
id             0
nametype       0
recclass       0
mass (g)       0
fall           0
year           0
reclat         0
reclong        0
GeoLocation    0
dtype: int64

In [19]:
print("dimensiones con nulls =", df_3.shape)
print("dimensiones sin nulls =", df_3_nn.shape)

dimensiones con nulls = (45716, 10)
dimensiones sin nulls = (38115, 10)


In [50]:
 df_3_nn

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)"
5,Adhi Kot,379,Valid,EH4,4239.0,Fell,01/01/1919 12:00:00 AM,32.10000,71.80000,"(32.1, 71.8)"
6,Adzhi-Bogdo (stone),390,Valid,LL3-6,910.0,Fell,01/01/1949 12:00:00 AM,44.83333,95.16667,"(44.83333, 95.16667)"
7,Agen,392,Valid,H5,30000.0,Fell,01/01/1814 12:00:00 AM,44.21667,0.61667,"(44.21667, 0.61667)"
8,Aguada,398,Valid,L6,1620.0,Fell,01/01/1930 12:00:00 AM,-31.60000,-65.23333,"(-31.6, -65.23333)"
9,Aguila Blanca,417,Valid,L,1440.0,Fell,01/01/1920 12:00:00 AM,-30.86667,-64.55000,"(-30.86667, -64.55)"


In [51]:
df_3_nn.columns

Index(['name', 'id', 'nametype', 'recclass', 'mass (g)', 'fall', 'year',
       'reclat', 'reclong', 'GeoLocation'],
      dtype='object')

In [ ]:
data_set["Platform"].value_counts()

### Near Earth Comets Orbital Elements

In [14]:
df_4 #Near-Earth_Comets_-_Orbital_Elements

,Object,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d),ref,Object_name
0,P/2004 R1 (McNaught),54629,2455248.548,0.682527,4.894556,0.626838,295.985450,0.986192,5.23,5.48,0.027011,NaN,NaN,NaN,NaN,20,P/2004 R1 (McNaught)
1,P/2008 S1 (Catalina-McNaught),55101,2454741.329,0.666313,15.100746,203.649023,111.392003,1.190642,5.95,6.74,0.194101,NaN,NaN,NaN,NaN,13,P/2008 S1 (Catalina-McNaught)
2,1P/Halley,49400,2446467.395,0.967143,162.262691,111.332485,58.420081,0.585978,35.08,75.32,0.063782,2.700000e-10,1.550000e-10,NaN,NaN,J863/77,1P/Halley
3,2P/Encke,56870,2456618.204,0.848268,11.779995,186.540346,334.569806,0.336092,4.09,3.30,0.173092,1.580000e-10,-5.050000e-12,NaN,NaN,74,2P/Encke
4,3D/Biela,-9480,2390514.115,0.751299,13.216400,221.658800,250.669000,0.879073,6.19,6.65,0.000518,3.900000e-09,-2.540000e-10,NaN,NaN,IAUCAT03,3D/Biela
5,5D/Brorsen,7440,2407439.534,0.809796,29.382100,14.946800,102.967600,0.589847,5.61,5.46,0.366559,1.270000e-08,1.340000e-09,NaN,NaN,IAUCAT03,5D/Brorsen
6,7P/Pons-Winnecke,56981,2457053.028,0.637528,22.335015,172.506161,93.416327,1.239215,5.60,6.32,0.224191,7.860000e-11,-1.150000e-11,-1.530000e-10,131.00,K088/14,7P/Pons-Winnecke
7,8P/Tuttle,54374,2454492.526,0.819800,54.983185,207.509246,270.341652,1.027117,10.37,13.61,0.095310,6.890000e-10,1.370000e-10,-4.590000e-10,NaN,K074/27,8P/Tuttle
8,12P/Pons-Brooks,35000,2434885.381,0.954812,74.176894,199.028469,255.891144,0.773667,33.47,70.85,0.187300,-1.020000e-09,-2.710000e-10,NaN,NaN,15,12P/Pons-Brooks
9,13P/Olbers,35760,2435643.635,0.930297,44.609886,64.641207,86.103128,1.178451,32.64,69.52,0.477199,9.450000e-09,6.490000e-10,NaN,NaN,10,13P/Olbers


In [17]:
#Se buscan los Null
np.sum(df_4.isnull())

Object           0
Epoch (TDB)      0
TP (TDB)         0
e                0
i (deg)          0
w (deg)          0
Node (deg)       0
q (AU)           0
Q (AU)           0
P (yr)           0
MOID (AU)        0
A1 (AU/d^2)    122
A2 (AU/d^2)    119
A3 (AU/d^2)    150
DT (d)         156
ref              0
Object_name      0
dtype: int64